# Checking Results

In [ ]:
import pandas as pd
import pygmo as pg
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
configs = pd.read_csv('./results_no_cv_HO_29_10')

In [ ]:
configs

In [ ]:
pareto_index = pg.non_dominated_front_2d(configs[['rmse_test', 'std_test']])

In [ ]:
pareto_configs = configs.iloc[pareto_index]

In [ ]:
pareto_configs

In [ ]:
pareto_configs.iloc[1].net_cfg

In [ ]:
x = pareto_configs['rmse_test'].values
y = pareto_configs['std_test'].values
plt.fig
plt.scatter(x, y)
plt.xlabel('rmse_test')
plt.ylabel('std_test')

# for i, txt in enumerate(pareto_configs.index):
#     plt.annotate(txt, (x[i], y[i]))

In [ ]:
pareto_configs

In [ ]:
sns.set_theme()

plt.figure(figsize=(15,10))
sns.scatterplot(data=pareto_configs, x='rmse_test', y='std_test')
plt.title('Pareto Front')

In [ ]:
pareto_configs.loc[186].net_cfg

# Checking Output

In [ ]:
from sklearn.utils import check_random_state

In [ ]:
check_random_state(42)

In [ ]:
np.random.seed(42)
np.random.choice(range(1,100), replace=False, size=10)


In [ ]:
from sklearn.model_selection import KFold


In [ ]:
test = [14, 20, 51, 60, 71, 74, 82, 86, 91, 92]
train = [x for x in range(0,100) if x not in test]


In [ ]:
train

In [ ]:
# np.random.get_state(np.random.seed(42))

In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"

import tensorflow as tf
from preprocessing import build_data
import pandas as pd
import math
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np


from activations import Activate
from losses import CustomLoss
from preprocessing import build_data


import main



In [2]:
def weibull_mean(alpha, beta):
    return alpha * math.gamma(1 + 1/beta)

In [3]:
# model, train_results_df, test_results_df, test_x_orig, test_y_orig, scaler, train_x, test_x, total = main.main()
model = main.main()

/home/kefalasm/Projects/rul_sa/mipego/mipego.py:215: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  self._levels = np.array([self._space.bounds[i] for i in self._space.id_N]) # levels for discrete variable


{'max_time': 100, 'lr': '0.001', 'num_rec': 3, 'neuron_0': 100, 'activation_0': 'tanh', 'dropout_0': 0.25, 'recurrent_dropout_0': 0.25, 'neuron_1': 50, 'activation_1': 'tanh', 'dropout_1': 0.25, 'recurrent_dropout_1': 0.25, 'neuron_2': 20, 'activation_2': 'tanh', 'dropout_2': 0.25, 'recurrent_dropout_2': 0.25, 'final_activation_0': 'exp', 'final_activation_1': 'softplus', 'percentage': 50, 'rul': 115, 'rul_style': 'nonlinear', 'batch': '128'}
(16340, 26)
(10312, 26)
(100,)


100%|██████████| 100/100 [00:00<00:00, 12591.73it/s]


train_x (16340, 100, 17) train_y (16340, 1) test_x (100, 100, 17) test_y (100, 1)
Model: "weibull_params"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 17)]         0         
_________________________________________________________________
masking (Masking)            (None, 100, 17)           0         
_________________________________________________________________
gru (GRU)                    (None, 100, 100)          35700     
_________________________________________________________________
gru_1 (GRU)                  (None, 100, 50)           22800     
_________________________________________________________________
gru_2 (GRU)                  (None, 20)                4320      
_________________________________________________________________
dense (Dense)                (None, 2)                 42        
____________________________________

/home/kefalasm/rul_nn/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Epoch 1/5
128/128 [==============================] - 119s 889ms/step - loss: 2.0580 - val_loss: 0.2433
Epoch 2/5
128/128 [==============================] - 113s 884ms/step - loss: 0.2163 - val_loss: -0.0899
Epoch 3/5
128/128 [==============================] - 114s 892ms/step - loss: -0.1457 - val_loss: -0.4530
Epoch 4/5
128/128 [==============================] - 114s 893ms/step - loss: -0.4130 - val_loss: -0.5514
Epoch 5/5
128/128 [==============================] - 115s 901ms/step - loss: -0.5697 - val_loss: -0.6276


In [4]:
model

In [10]:
# model.save('./toy_model')

In [9]:
# new_model = tf.keras.models.load_model('./toy_model', custom_objects={"CustomLoss": CustomLoss, "Activate": Activate})

In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="-1"  #"1,2,3,4,5"  # uncomment in case running ONLY on CPU is required

import tensorflow as tf
tf.random.set_seed(42)
from tensorflow.keras import backend as k
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import History
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

from activations import Activate
from losses import CustomLoss
from preprocessing import build_data

import numpy as np
import pandas as pd
import math
from datetime import datetime

from sklearn import pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [2]:
net_cfg = {
     "max_time": 100,
     "lr": "0.001",
     "num_rec": 3,
     "neuron_0": 100,
     "activation_0": "tanh",
     "dropout_0": 0.25,
     "recurrent_dropout_0": 0.25,
     "neuron_1": 50,
     "activation_1": "tanh",
     "dropout_1": 0.25,
     "recurrent_dropout_1": 0.25,
     "neuron_2": 20,
     "activation_2": "tanh",
     "dropout_2": 0.25,
     "recurrent_dropout_2": 0.25,
     "final_activation_0": "exp",
     "final_activation_1": "softplus",
     "percentage": 50,
     "rul": 115,
     "rul_style": "nonlinear",
     "batch": "128",
}

In [3]:
cfg = {
        "cv": 10,
        "shuffle": True,
        "random_state": 21,
        "mask_value": -99,
        "reps": 30,
        "epochs": 5,
        "batches": 64,
        "in_reps": 10,
    }

In [4]:
def weibull_mean(alpha, beta):
    return alpha * math.gamma(1 + 1/beta)

In [9]:
def network(train_X, train_y, net_cfg, cfg):
    k.set_epsilon(1e-10)
    history = History()
    nan_terminator = callbacks.TerminateOnNaN()
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss')
    early_stopping = callbacks.EarlyStopping(monitor='loss', patience=5)
    # checkpoint_filepath = './saved_models/cp-{epoch:04d}.ckpt'
    # checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='loss', verbose=1)
    # logdir="logs/test1" #+ datetime.now().strftime("%Y%m%d-%H%M%S")
    # tensorboard = callbacks.TensorBoard(log_dir=logdir)

    window = train_X.shape[1]
    n_features = train_X.shape[2]

    inputs = keras.Input(shape=(window, n_features))
    masking_layer = keras.layers.Masking(mask_value=cfg['mask_value'])(inputs)

    # recurrent layers
    if net_cfg['num_rec'] > 1:
        for i in np.arange(net_cfg['num_rec']-1):
            masking_layer = keras.layers.GRU(net_cfg['neuron_'+str(i)], activation=net_cfg['activation_'+str(i)],
                                    dropout=net_cfg['dropout_'+str(i)],
                                    recurrent_dropout=net_cfg['recurrent_dropout_'+str(i)],
                                    return_sequences=True)(masking_layer)
    last = i + 1
    gru_last = keras.layers.GRU(net_cfg['neuron_'+str(last)], activation=net_cfg['activation_'+str(last)],
                                dropout=net_cfg['dropout_'+str(last)],
                                recurrent_dropout=net_cfg['recurrent_dropout_'+str(last)],
                                return_sequences=False)(masking_layer)

    dense_1 = keras.layers.Dense(2)(gru_last)
    custom_activation = Activate(net_cfg=net_cfg)
    outputs = keras.layers.Activation(custom_activation)(dense_1)

    model = keras.Model(inputs=inputs, outputs=outputs, name="weibull_params")

    # rmse = tf.keras.metrics.RootMeanSquaredError()
    model.compile(loss=CustomLoss(kind='continuous', reduce_loss=True), optimizer=Adam(lr=eval(net_cfg['lr']),
                                                                                       clipvalue=0.5))
    model.summary()  # uncomment for debugging

    batch_size=eval(net_cfg['batch'])
    model.fit(train_X, train_y,
              epochs=cfg['epochs'],
              batch_size=eval(net_cfg['batch']),
              verbose=1,
              callbacks=[nan_terminator, history, reduce_lr, early_stopping],#, checkpoint, tensorboard],  # , tensorboard
              workers=32)

    return model, history



In [10]:
def load_data():

    np.random.seed(42)

    id_col = 'unit_number'
    time_col = 'time'
    feature_cols = ['op_setting_1', 'op_setting_2', 'op_setting_3'] + ['sensor_measurement_{}'.format(x) for x in
                                                                       range(1, 22)]
    column_names = [id_col, time_col] + feature_cols

    train_x_orig = pd.read_csv('./DataSets/CMAPSS/train_FD001.csv', header=None, sep='\s+', decimal=".")
    train_x_orig.columns = column_names

    test_x_orig = pd.read_csv('./DataSets/CMAPSS/test_FD001.csv', header=None, sep='\s+', decimal=".")
    test_x_orig.columns = column_names

    test_y_orig = pd.read_csv('./DataSets/CMAPSS/RUL_FD001.csv', header=None, names=['T'])

    # Make engine numbers and days zero-indexed
    train_x_orig.iloc[:, 0:2] -= 1
    test_x_orig.iloc[:, 0:2] -= 1

    train_idx = np.random.choice(range(train_x_orig.unit_number.unique().max()+1), replace=False, size=80)  # selecting 80 units for training
    train_idx.sort()

    vld_idx = np.array([x for x in range(train_x_orig.unit_number.unique().max()+1) if x not in train_idx])  # remaining are validation indices

    train_or = train_x_orig[train_x_orig.unit_number.isin(train_idx)]  # training data
    train_or.reset_index(drop=True, inplace=True)
    vld = train_x_orig[train_x_orig.unit_number.isin(vld_idx)]  # validation data
    vld.reset_index(drop=True, inplace=True)


    # Truncating the validation data randomly 5 times each
    vld_trunc = []
    test_y = []
    max_cycle = []
    test_index = []  # for debugging purposes
    temp_or_test_cycles = []
    counter = -1

    for i in set(vld.unit_number.unique()):
        # print(f'unit number is {i}')
        for j in range(1,6):  # 5 truncations per instance
            counter += 1
            np.random.seed(i*j)
            temp_df = vld[vld.unit_number == i]
            temp_df.reset_index(drop=True, inplace=True)  # important
            length = temp_df.shape[0]
            # print(length)
            temp_or_test_cycles.append(length)
            level = np.random.choice(np.arange(5, 96), 1)[0]
            r = np.int(length * (1 - level / 100))
            # test_index.append(X_test_or[X_test_or.unit_number == i].index.tolist()[
            #                     :r + 1])  # check this with train_x_orig instead of X_test_or (probably it's the same)
            temp_df = temp_df.truncate(after=r)
            # print(temp_df.shape[0])
            # print('\n')
            temp_df['unit_number'] = np.repeat(counter, temp_df.shape[0])
            vld_trunc.append(temp_df)
            max_cycle.append(length)


    # test_index = [item for sublist in test_index for item in sublist]

    vld_trunc = pd.concat(vld_trunc)
    vld_trunc.reset_index(drop=True, inplace=True)
    # print(f'max len per unit is {max_cycle}')


    # Pre-processing data
    scaler = pipeline.Pipeline(steps=[
        ('minmax', MinMaxScaler(feature_range=(-1, 1))),
        ('remove_constant', VarianceThreshold())])

    train = train_or.copy()
    train = np.concatenate([train[['unit_number', 'time']], scaler.fit_transform(train[feature_cols])], axis=1)

    train_x, train_y = build_data(units=train[:, 0], time=train[:, 1], x=train[:, 2:], max_time=net_cfg['max_time'],
                                    is_test=False, mask_value=cfg['mask_value'],
                                    original_data=None, net_cfg = net_cfg, label=net_cfg['rul_style'])

    vld = vld_trunc.copy()
    vld = np.concatenate([vld[['unit_number', 'time']], scaler.transform(vld[feature_cols])], axis=1)
    test_x, test_y = build_data(units=vld[:, 0],
        time=vld[:, 1],
        x=vld[:, 2:],
        max_time=net_cfg["max_time"],
        is_test=True,
        mask_value=cfg["mask_value"],
        original_data=np.array(max_cycle),
        net_cfg=net_cfg,
        label=net_cfg["rul_style"],
    )

    return train_x, train_y, test_x, test_y, train_or, vld_trunc

In [11]:
train_x, train_y, test_x, test_y, train, test = load_data()
print('train_x', train_x.shape, 'train_y', train_y.shape, 'test_x', test_x.shape, 'test_y', test_y.shape, 'train', train.shape, 'test', test.shape)



100%|██████████| 100/100 [00:00<00:00, 18112.47it/s]

train_x (16340, 100, 17) train_y (16340, 1) test_x (100, 100, 17) test_y (100, 1) train (16340, 26) test (10312, 26)


In [12]:
k.clear_session()
try:
    del model
except:
    pass

model, history = network(train_x, train_y, net_cfg, cfg)

/home/kefalasm/rul_nn/lib/python3.8/site-packages/tensorflow/python/keras/optimizer_v2/optimizer_v2.py:374: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  warnings.warn(


Model: "weibull_params"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 100, 17)]         0         
_________________________________________________________________
masking (Masking)            (None, 100, 17)           0         
_________________________________________________________________
gru (GRU)                    (None, 100, 100)          35700     
_________________________________________________________________
gru_1 (GRU)                  (None, 100, 50)           22800     
_________________________________________________________________
gru_2 (GRU)                  (None, 20)                4320      
_________________________________________________________________
dense (Dense)                (None, 2)                 42        
_________________________________________________________________
activation (Activation)      (None, 2)              

INFO:tensorflow:Assets written to: ./saved_models/cp-0001.ckpt/assets


INFO:tensorflow:Assets written to: ./saved_models/cp-0001.ckpt/assets


Epoch 2/5
128/128 [==============================] - 48s 374ms/step - loss: 0.2623

Epoch 00002: saving model to ./saved_models/cp-0002.ckpt


INFO:tensorflow:Assets written to: ./saved_models/cp-0002.ckpt/assets


INFO:tensorflow:Assets written to: ./saved_models/cp-0002.ckpt/assets


Epoch 3/5
 40/128 [========>.....................] - ETA: 32s - loss: 0.0204

KeyboardInterrupt: 

In [ ]:
for i in range(cfg["reps"]):
        tf.random.set_seed(i)
        train_predict = model(train_x, training=True).numpy()
        a, b = train_predict[:, 0], train_predict[:, 1]

        res = []
        for j in range(a.shape[0]):
            res.append(a[j] * np.random.weibull(b[j], cfg["in_reps"]))
        total_res_train.append(res)
    total_res_train = np.array(total_res_train)

    train_predict = np.mean(total_res_train, axis=(0, 2))
    train_predict = np.reshape(train_predict, (train_x.shape[0], 1))
    train_result = np.concatenate((train_y, train_predict), axis=1)
    train_results_df = pd.DataFrame(train_result, columns=["T", "predicted_mu"])
    train_results_df["unit_number"] = train_x_orig["unit_number"].to_numpy()
    train_results_df["time"] = train_x_orig["time"].to_numpy()

In [ ]:
samples = []
for i in range(train_x.shape[0]):
            tf.random.set_seed(i)
            train_predict = model(train_x[i].reshape(1, train_x[i].shape[0], train_x[i].shape[1]), training=True).numpy()
            a, b = train_predict[:,0], train_predict[:,1]
            samples.append(b.reshape(-1,1)*np.random.weibull(a.reshape(-1,1), size=(a.shape[0], 30)))


In [ ]:
plt.plot(model.history.history['loss'],    label='training')
plt.plot(model.history.history['val_loss'],label='validation')
plt.title('loss')
plt.legend()

In [ ]:
def plot_weibull_predictions(results_df, unit):

#     fig, axarr = plt.subplots(3, figsize=(30,20))
    
    plt.figure(figsize=(30,10))

    t=np.arange(0,400)

    palette = sns.color_palette("RdBu_r", results_df.shape[0] + 1)
    color_dict = dict(enumerate(palette))

#     ax=axarr[0]
    
#     median_predictions = weibull_median(results_df['alpha'], results_df['beta'])
    mean_predictions =  results_df['predicted_mu'] # results_df[['alpha', 'beta']].apply(lambda row: weibull_mean(row[0], row[1]), axis=1)
    std_plus = results_df['predicted_std+']
    std_minus = results_df['predicted_std-']

    x = results_df['time']
    
    plt.plot(x, results_df['T'], label='survival_time', color='black')

#     ax.plot(x, median_predictions, label='median_prediction')
    plt.plot(x, mean_predictions, label='mean_prediction')
#     ax.plot(x, mode_predictions, label='mode_prediction')
#     ax.set_title('MAP prediction Vs. true')
    plt.plot(x, std_plus, 'g-')
    plt.plot(x, std_minus, 'g-')
    

    plt.legend()
    
#     ax=axarr[1]
#     sns.distplot(results_df['T'] - mode_predictions, ax=ax)
#     ax.set_title('Error')

#     ax.plot(x, results_df['alpha'], label='alpha')
#     ax.legend()
    
    
#     ax = axarr[2]
#     ax.plot(x, results_df['beta'], label='beta')
#     ax.legend()
    
#     ax.legend(loc='center left', bbox_to_anchor=(1, 0.5))
    plt.suptitle(unit)
    plt.show()

In [ ]:
train_results_df

In [ ]:
test_results_df

In [ ]:
test_results_df['T'].unique()

In [ ]:
net_cfg = {"num_rec": 2, "max_time": 25, "neuron_0": 74, "neuron_1": 71, "neuron_2": 78, "neuron_3": 68,
     "activation_0": "tanh", "activation_1": "sigmoid", "activation_2": "sigmoid", "activation_3": "tanh",
      "dropout_0": 0.04198965650706104, "dropout_1": 0.6518949855946009, "dropout_2": 0.5134433415117658,
       "dropout_3": 0.44181882048621723, "recurrent_dropout_0": 0.011379281378212352, "recurrent_dropout_1": 0.07877843876273939,
        "recurrent_dropout_2": 0.25685072170110057, "recurrent_dropout_3": 0.16640448683710898, "final_activation_0": "softplus",
         "final_activation_1": "softplus", "percentage": 63, "rul": 125, "rul_style": "nonlinear", "lr": 0.021745779733326226, "batch": "256"}

In [ ]:
id_col = 'unit_number'
time_col = 'time'
max_time = 32
mask_value = -99

feature_cols = ['op_setting_1', 'op_setting_2', 'op_setting_3'] + ['sensor_measurement_{}'.format(x) for x in
                                                                    range(1, 22)]
column_names = [id_col, time_col] + feature_cols
test_or = test_x_orig.copy()
test_or = np.concatenate([test_or[['unit_number', 'time']], scaler.transform(test_or[feature_cols])], axis=1)


# Preparing data for the RNN (numpy arrays)
test_or, _ = build_data(units=test_or[:, 0], time=test_or[:, 1], x=test_or[:, 2:], max_time=net_cfg['max_time'],
                                is_test=True, mask_value=-99,
                                original_data=np.repeat(200, test_or.shape[0]), net_cfg = net_cfg, label=net_cfg['rul_style'])


In [ ]:
reps = 30
test_predict_1_or = []
test_predict_2_or = []
for _ in range(reps):
    test_predict_or = model(test_or, training=True).numpy()
#     print(test_predict_or.shape)
#     break
    test_predict_1_or.append(test_predict_or[:,0].reshape(test_predict_or[:,0].shape[0], 1))
    test_predict_2_or.append(test_predict_or[:,1].reshape(test_predict_or[:,1].shape[0], 1))

test_predict_1_mean_or = np.average(np.hstack(test_predict_1_or), axis=1)
test_predict_2_mean_or = np.average(np.hstack(test_predict_2_or), axis=1)
test_predict_1_mean_or = test_predict_1_mean_or.reshape(test_predict_1_mean_or.shape[0],1)
test_predict_2_mean_or = test_predict_2_mean_or.reshape(test_predict_2_mean_or.shape[0],1)
test_predict_1_std_or = np.std(np.hstack(test_predict_1_or), axis=1)
test_predict_2_std_or = np.std(np.hstack(test_predict_2_or), axis=1)
test_predict_1_std_or = test_predict_1_std_or.reshape(test_predict_1_std_or.shape[0],1)
test_predict_2_std_or = test_predict_2_std_or.reshape(test_predict_2_std_or.shape[0],1)


test_predict_or = np.hstack([test_predict_1_mean_or, test_predict_2_mean_or, 
                           test_predict_1_std_or, test_predict_2_std_or])    

test_predict_or = np.resize(test_predict_or, (test_or.shape[0], 4))  # changed from 2 to 4
test_result_or = np.concatenate((test_y_orig, test_predict_or), axis=1)
test_results_df_or = pd.DataFrame(test_result_or, columns=['T',   'mean_alpha', 'mean_beta', 'std_alpha', 'std_beta']) # (add 'E' for event)
#     test_results_df['unit_number'] = train_x_orig.iloc[test_index]['unit_number'].to_numpy() # Note the train_x_orig
#     test_results_df['time'] = train_x_orig.iloc[test_index]['time'].to_numpy()

test_results_df_or['predicted_mu'] = test_results_df_or[['mean_alpha', 'mean_beta']].apply(lambda row: weibull_mean(row[0], row[1]), axis=1)
test_results_df_or['predicted_std+'] = test_results_df_or[['mean_alpha', 'mean_beta', 'std_alpha', 
                                                       'std_beta']].apply(lambda row: weibull_mean(row[0]+1.96*row[2]/np.sqrt(reps), 
                                                                                                   row[1]+1.96*row[3]/np.sqrt(reps)), axis=1)
test_results_df_or['predicted_std-'] = test_results_df_or[['mean_alpha', 'mean_beta', 'std_alpha', 
                                                           'std_beta']].apply(lambda row: weibull_mean(row[0]-1.96*row[2]/np.sqrt(reps),
                                                                                                       row[1]-1.96*row[3]/np.sqrt(reps)), axis=1)




# Old way
# test_predict_or = model(test_or, training=True).numpy()  # equivalent to model.predict(test_or) but with training=False
# test_predict_or = np.resize(test_predict_or, (test_or.shape[0], 2))
# test_result_or = np.concatenate((test_y_orig, test_predict_or), axis=1)
# test_results_df_or = pd.DataFrame(test_result_or, columns=['T', 'alpha', 'beta'])
# test_results_df_or['unit_number'] = np.arange(1, test_results_df_or.shape[0]+1)

# test_results_df_or['predicted_mu'] = test_results_df_or[['alpha', 'beta']].apply(lambda row: weibull_mean(row[0], row[1]), axis=1)

In [ ]:
test_results_df_or

In [ ]:
#1 - 10 epochs
np.sqrt(mean_squared_error(test_results_df_or['predicted_mu'], test_results_df_or['T']))

In [ ]:
#1 - 50 epochs
np.sqrt(mean_squared_error(test_results_df_or['predicted_mu'], test_results_df_or['T']))

In [ ]:
#2 - 21/50 epochs
np.sqrt(mean_squared_error(test_results_df_or['predicted_mu'], test_results_df_or['T']))

In [ ]:
for unit_number, grp in train_results_df.groupby('unit_number'):
    plot_weibull_predictions(grp, unit_number)
    break

In [ ]:
for unit_number, grp in test_results_df.groupby('unit_number'):
    plot_weibull_predictions(grp, unit_number)
    break

# Training full model on the best HP

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1,2,3,4,5"  # uncomment in case running ONLY on CPU is required

import tensorflow as tf
tf.random.set_seed(42)
from tensorflow.keras import backend as k
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import History
from tensorflow import keras
from tensorflow.keras.optimizers import Adam

from activations import Activate
from losses import CustomLoss
from preprocessing import build_data

import numpy as np
import pandas as pd
import math
from datetime import datetime

from sklearn import pipeline
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import MinMaxScaler

In [ ]:
net_cfg = {"num_rec": 3, "max_time": 26, "neuron_0": 73, "neuron_1": 71, "neuron_2": 82, "neuron_3": 82, "activation_0": "tanh",
     "activation_1": "sigmoid", "activation_2": "sigmoid", "activation_3": "tanh", "dropout_0": 0.06943171652267692,
      "dropout_1": 0.12639579059484615, "dropout_2": 0.3822443511564662, "dropout_3": 0.4580962846531429,
       "recurrent_dropout_0": 0.3280089650917844, "recurrent_dropout_1": 0.69930466502713, "recurrent_dropout_2": 0.24506744915217923,
        "recurrent_dropout_3": 0.7699919737017498, "final_activation_0": "exp", "final_activation_1": "softplus", "percentage": 73,
        "rul": 121, "rul_style": "nonlinear", "lr": 0.005357912753227542, "batch": "128"}


In [ ]:
cfg = {'cv': 10, 'shuffle': True,
'random_state': 21,
'mask_value': -99,
'reps': 30,
'epochs': 100,
'batches': 64}

In [ ]:
def weibull_mean(alpha, beta):
    return alpha * math.gamma(1 + 1/beta)

In [ ]:
def network(train_X, train_y, net_cfg, cfg):
    k.set_epsilon(1e-10)
    history = History()
    nan_terminator = callbacks.TerminateOnNaN()
    reduce_lr = callbacks.ReduceLROnPlateau(monitor='loss')
    early_stopping = callbacks.EarlyStopping(monitor='loss', patience=5)
    checkpoint_filepath = './saved_models/cp-{epoch:04d}.ckpt'
    checkpoint = callbacks.ModelCheckpoint(filepath=checkpoint_filepath, monitor='loss', verbose=1)
    logdir="logs/test1" #+ datetime.now().strftime("%Y%m%d-%H%M%S")
    tensorboard = callbacks.TensorBoard(log_dir=logdir)

    window = train_X.shape[1]
    n_features = train_X.shape[2]

    inputs = keras.Input(shape=(window, n_features))
    masking_layer = keras.layers.Masking(mask_value=cfg['mask_value'])(inputs)

    # recurrent layers
    if net_cfg['num_rec'] > 1:
        for i in np.arange(net_cfg['num_rec']-1):
            masking_layer = keras.layers.GRU(net_cfg['neuron_'+str(i)], activation=net_cfg['activation_'+str(i)],
                                    dropout=net_cfg['dropout_'+str(i)],
                                    recurrent_dropout=net_cfg['recurrent_dropout_'+str(i)],
                                    return_sequences=True)(masking_layer)
    last = i + 1
    gru_last = keras.layers.GRU(net_cfg['neuron_'+str(last)], activation=net_cfg['activation_'+str(last)],
                                dropout=net_cfg['dropout_'+str(last)],
                                recurrent_dropout=net_cfg['recurrent_dropout_'+str(last)],
                                return_sequences=False)(masking_layer)

    dense_1 = keras.layers.Dense(2)(gru_last)
    custom_activation = Activate(net_cfg=net_cfg)
    outputs = keras.layers.Activation(custom_activation)(dense_1)

    model = keras.Model(inputs=inputs, outputs=outputs, name="weibull_params")

    # rmse = tf.keras.metrics.RootMeanSquaredError()
    model.compile(loss=CustomLoss(kind='continuous', reduce_loss=True), optimizer=Adam(lr=net_cfg['lr'],
                                                                                       clipvalue=0.5))
    model.summary()  # uncomment for debugging

    batch_size=eval(net_cfg['batch'])
    model.fit(train_X, train_y,
              epochs=cfg['epochs'],
              batch_size=eval(net_cfg['batch']),
              verbose=1,
              callbacks=[nan_terminator, history, reduce_lr, early_stopping, checkpoint],#, tensorboard],  # , tensorboard
              workers=32)

    return model, history



In [ ]:
# %load_ext tensorboard


In [ ]:
# %tensorboard --logdir=logdir --port 8889


In [ ]:
def load_data():

    np.random.seed(42)

    id_col = 'unit_number'
    time_col = 'time'
    feature_cols = ['op_setting_1', 'op_setting_2', 'op_setting_3'] + ['sensor_measurement_{}'.format(x) for x in
                                                                       range(1, 22)]
    column_names = [id_col, time_col] + feature_cols

    train_x_orig = pd.read_csv('./DataSets/CMAPSS/train_FD001.csv', header=None, sep='\s+', decimal=".")
    train_x_orig.columns = column_names

    test_x_orig = pd.read_csv('./DataSets/CMAPSS/test_FD001.csv', header=None, sep='\s+', decimal=".")
    test_x_orig.columns = column_names

    test_y_orig = pd.read_csv('./DataSets/CMAPSS/RUL_FD001.csv', header=None, names=['T'])

    # Make engine numbers and days zero-indexed
    train_x_orig.iloc[:, 0:2] -= 1
    test_x_orig.iloc[:, 0:2] -= 1

    # Pre-processing data
    scaler = pipeline.Pipeline(steps=[
        ('minmax', MinMaxScaler(feature_range=(-1, 1))),
        ('remove_constant', VarianceThreshold())])

    train = train_x_orig.copy()
    train = np.concatenate([train[['unit_number', 'time']], scaler.fit_transform(train[feature_cols])], axis=1)

    train_x, train_y = build_data(units=train[:, 0], time=train[:, 1], x=train[:, 2:], max_time=net_cfg['max_time'],
                                    is_test=False, mask_value=cfg['mask_value'],
                                    original_data=None, net_cfg = net_cfg, label=net_cfg['rul_style'])

    test_or = test_x_orig.copy()
    test_or = np.concatenate([test_or[['unit_number', 'time']], scaler.transform(test_or[feature_cols])], axis=1)


    # Preparing data for the RNN (numpy arrays)
    test_or, _ = build_data(units=test_or[:, 0], time=test_or[:, 1], x=test_or[:, 2:], max_time=net_cfg['max_time'],
                                    is_test=True, mask_value=-99,
                                    original_data=np.repeat(200, test_or.shape[0]), net_cfg = net_cfg, label=net_cfg['rul_style'])

    

    return train_x, train_y, test_or, test_y_orig, train_x_orig


In [ ]:
train_x, train_y, test_x, test_y, train_x_orig = load_data()


In [ ]:
print('train_x', train_x.shape, 'train_y', train_y.shape, 'test_x', test_x.shape, 'test_y', test_y.shape)


In [ ]:
k.clear_session()
try:
    del model
except:
    pass

model, history = network(train_x, train_y, net_cfg, cfg)

In [ ]:
model = tf.keras.models.load_model('../autokeras_rul/best_model_autokeras_with_vld') #, custom_objects={"CustomLoss": CustomLoss, "Activate": Activate})

In [ ]:
model.summary()

In [ ]:
model.evaluate(train_x, train_y)

In [ ]:
model.evaluate(test_x, test_y)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(model.history.history['loss'],    label='training')
plt.title('loss')
plt.legend()

In [ ]:
reps = 30
test_predict_1_or = []
test_predict_2_or = []
for _ in range(reps):
    test_predict_or = model(test_x, training=True).numpy()
#     print(test_predict_or.shape)
#     break
    test_predict_1_or.append(test_predict_or[:,0].reshape(test_predict_or[:,0].shape[0], 1))
    test_predict_2_or.append(test_predict_or[:,1].reshape(test_predict_or[:,1].shape[0], 1))

test_predict_1_mean_or = np.average(np.hstack(test_predict_1_or), axis=1)
test_predict_2_mean_or = np.average(np.hstack(test_predict_2_or), axis=1)
test_predict_1_mean_or = test_predict_1_mean_or.reshape(test_predict_1_mean_or.shape[0],1)
test_predict_2_mean_or = test_predict_2_mean_or.reshape(test_predict_2_mean_or.shape[0],1)
test_predict_1_std_or = np.std(np.hstack(test_predict_1_or), axis=1)
test_predict_2_std_or = np.std(np.hstack(test_predict_2_or), axis=1)
test_predict_1_std_or = test_predict_1_std_or.reshape(test_predict_1_std_or.shape[0],1)
test_predict_2_std_or = test_predict_2_std_or.reshape(test_predict_2_std_or.shape[0],1)


test_predict_or = np.hstack([test_predict_1_mean_or, test_predict_2_mean_or, 
                           test_predict_1_std_or, test_predict_2_std_or])    

test_predict_or = np.resize(test_predict_or, (test_x.shape[0], 4))  # changed from 2 to 4
test_result_or = np.concatenate((test_y, test_predict_or), axis=1)
test_results_df_or = pd.DataFrame(test_result_or, columns=['T',   'mean_alpha', 'mean_beta', 'std_alpha', 'std_beta']) # (add 'E' for event)
#     test_results_df['unit_number'] = train_x_orig.iloc[test_index]['unit_number'].to_numpy() # Note the train_x_orig
#     test_results_df['time'] = train_x_orig.iloc[test_index]['time'].to_numpy()

test_results_df_or['predicted_mu'] = test_results_df_or[['mean_alpha', 'mean_beta']].apply(lambda row: weibull_mean(row[0], row[1]), axis=1)
# test_results_df_or['predicted_std+'] = test_results_df_or[['mean_alpha', 'mean_beta', 'std_alpha', 
#                                                        'std_beta']].apply(lambda row: weibull_mean(row[0]+1.96*row[2]/np.sqrt(reps), 
#                                                                                                    row[1]+1.96*row[3]/np.sqrt(reps)), axis=1)
# test_results_df_or['predicted_std-'] = test_results_df_or[['mean_alpha', 'mean_beta', 'std_alpha', 
#                                                            'std_beta']].apply(lambda row: weibull_mean(row[0]-1.96*row[2]/np.sqrt(reps),
#                                                                                                        row[1]-1.96*row[3]/np.sqrt(reps)), axis=1)




# Old way
# test_predict_or = model(test_or, training=True).numpy()  # equivalent to model.predict(test_or) but with training=False
# test_predict_or = np.resize(test_predict_or, (test_or.shape[0], 2))
# test_result_or = np.concatenate((test_y_orig, test_predict_or), axis=1)
# test_results_df_or = pd.DataFrame(test_result_or, columns=['T', 'alpha', 'beta'])
# test_results_df_or['unit_number'] = np.arange(1, test_results_df_or.shape[0]+1)

# test_results_df_or['predicted_mu'] = test_results_df_or[['alpha', 'beta']].apply(lambda row: weibull_mean(row[0], row[1]), axis=1)

In [ ]:
test_results_df_or

In [ ]:
train_predict_1 = []
train_predict_2 = []

success = True

for i in range(cfg['reps']):
    tf.random.set_seed(i)
    train_predict = model(train_x, training=True).numpy()
    train_predict_1.append(train_predict[:, 0].reshape(train_predict[:, 0].shape[0], 1))
    train_predict_2.append(train_predict[:, 1].reshape(train_predict[:, 1].shape[0], 1))

train_predict_1_mean = np.mean(np.hstack(train_predict_1), axis=1)
train_predict_2_mean = np.mean(np.hstack(train_predict_2), axis=1)
train_predict_1_mean = train_predict_1_mean.reshape(train_predict_1_mean.shape[0], 1)
train_predict_2_mean = train_predict_2_mean.reshape(train_predict_2_mean.shape[0], 1)
train_predict_1_std = np.std(np.hstack(train_predict_1), axis=1)
train_predict_2_std = np.std(np.hstack(train_predict_2), axis=1)
train_predict_1_std = train_predict_1_std.reshape(train_predict_1_std.shape[0], 1)
train_predict_2_std = train_predict_2_std.reshape(train_predict_2_std.shape[0], 1)

train_predict = np.hstack([train_predict_1_mean, train_predict_2_mean,
                            train_predict_1_std, train_predict_2_std])

train_predict = np.resize(train_predict, (train_x.shape[0], 4))  # changed from 2 to 4
train_result = np.concatenate((train_y, train_predict), axis=1)
train_results_df = pd.DataFrame(train_result, columns=['T', 'mean_alpha', 'mean_beta', 'std_alpha',
                                                        'std_beta'])  # (add 'E' for event)
train_results_df['unit_number'] = train_x_orig['unit_number'].to_numpy()
train_results_df['time'] = train_x_orig['time'].to_numpy()

train_results_df['predicted_mu'] = train_results_df[['mean_alpha', 'mean_beta']].apply(
    lambda row: weibull_mean(row[0], row[1]), axis=1)
# train_results_df['predicted_std+'] = train_results_df[['mean_alpha', 'mean_beta', 'std_alpha',
#                                                         'std_beta']].apply(
#     lambda row: weibull_mean(row[0] + 1.96 * row[2] / np.sqrt(cfg['reps']),
#                                 row[1] + 1.96 * row[3] / np.sqrt(cfg['reps'])), axis=1)
# train_results_df['predicted_std-'] = train_results_df[['mean_alpha', 'mean_beta', 'std_alpha',
#                                                         'std_beta']].apply(
#     lambda row: weibull_mean(row[0] - 1.96 * row[2] / np.sqrt(cfg['reps']),
#                                 row[1] - 1.96 * row[3] / np.sqrt(cfg['reps'])), axis=1)

In [ ]:
train_results_df

In [ ]:
np.sqrt(mean_squared_error(train_results_df['predicted_mu'], train_results_df['T']))

In [ ]:
r2_score(train_results_df['predicted_mu'], train_results_df['T'])

In [ ]:
np.sqrt(mean_squared_error(test_results_df_or['predicted_mu'], test_results_df_or['T']))

In [ ]:
r2_score(test_results_df_or['predicted_mu'], test_results_df_or['T'])